In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import optax
from flax import linen as nn
from flax.training import train_state
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

import jaxmetric as jm

In [2]:
key = jax.random.PRNGKey(0)

psi=0

poly = lambda x: x[0]**5 + x[1]**5 + x[2]**5 +x[3]**5 + x[4]**5 + psi* x[0]*x[1]*x[2]*x[3]*x[4]

points = np.array([[0.5457718 +0.42877685j, -0.96667378+0.00622714j, -0.2195527-0.23682649j, 0.01382411+0.10245113j, 1.+0.j]])        

In [3]:
points = jm.point_gen.generate_points_calabi_yau(key,(4,) ,poly ,100)

In [5]:
jm.metrics.cy_vol_form((4,),poly,points).mean()

Array(0.05799519, dtype=float32)

In [4]:
jm.metrics.aux_weight((4,),[1],poly,points).mean()

Array(0.02952868, dtype=float32)

In [6]:
jm.metrics.get_mass((4,),[1],poly,points).mean()

Array(2.49672, dtype=float32)